In [ ]:
%%markdown
# INFO-F-422 - Statistical Foundations of Machine Learning

### Python version of the "Predictions: Network-based methods" tutorial

This tutorial demonstrates how to perform regression with neural networks using Python and TensorFlow/Keras. It follows a similar structure and logic as the provided R tutorial, using the [Auto MPG dataset](https://archive.ics.uci.edu/ml/datasets/auto+mpg).

## Preliminaries

In this tutorial, you will learn how to:
- Load and preprocess the data
- Normalize the features
- Build linear and DNN (deep neural network) regression models
- Evaluate the models and visualize their performance

We will use TensorFlow, Keras, and other Python libraries for data handling and plotting.

## Setup


!pip install tensorflow pandas numpy matplotlib seaborn sklearn
```

```python
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
```

## The Auto MPG dataset

We use the Auto MPG dataset. The goal is to predict the fuel efficiency (miles per gallon, MPG) of a car from its characteristics.

**Data description:**
- mpg: Miles per gallon (target variable)
- cylinders
- displacement
- horsepower
- weight
- acceleration
- model_year
- origin
- car_name

### Load the data

In [ ]:
url = "http://archive.ics.uci.edu/ml/machine-learning-databases/auto-mpg/auto-mpg.data"
column_names = ["mpg","cylinders","displacement","horsepower","weight","acceleration","model_year","origin","car_name"]

raw_dataset = pd.read_csv(url, names=column_names,
                          na_values='?', comment='\t',
                          sep=' ', skipinitialspace=True)

dataset = raw_dataset.copy()
dataset.tail()

### Clean the data

In [ ]:
# Drop rows with missing values
dataset = dataset.dropna()

# The 'origin' column is categorical, so we convert it to one-hot encoding
dataset["origin"] = dataset["origin"].replace({1:"USA", 2:"Europe", 3:"Japan"})
dataset = pd.get_dummies(dataset, columns=["origin"], prefix="", prefix_sep="")

dataset = dataset.drop("car_name", axis=1)
dataset.tail()

## Split the data into training and test sets

In [ ]:
train_dataset, test_dataset = train_test_split(dataset, test_size=0.2, random_state=42)

train_features = train_dataset.drop("mpg", axis=1)
test_features = test_dataset.drop("mpg", axis=1)

train_labels = train_dataset["mpg"]
test_labels = test_dataset["mpg"]

## Inspect the data

In [ ]:
train_dataset.describe().transpose()

We see different scales for different features. Normalization is important.

## Normalization

We'll use a `tf.keras.layers.Normalization` layer to normalize our data.

In [ ]:
normalizer = layers.Normalization(axis=-1)
normalizer.adapt(np.array(train_features))

When we call the normalizer on the data, it will normalize each feature to have mean 0 and variance 1.

## Linear regression with a single input (Horsepower)

Let's first build a simple linear regression model that predicts `mpg` from the `horsepower` feature alone.

We will:
1. Extract the `horsepower` feature.
2. Create a `Normalization` layer for just this feature.
3. Build a linear model `mpg = w * horsepower + b`.

In [ ]:
horsepower = np.array(train_features["horsepower"])
horsepower_normalizer = layers.Normalization(input_shape=[1,], axis=None)
horsepower_normalizer.adapt(horsepower)

### Single-variable linear model

In [ ]:
horsepower_model = keras.Sequential([
    horsepower_normalizer,
    layers.Dense(units=1)
])

horsepower_model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.1),
    loss='mean_absolute_error'
)

history = horsepower_model.fit(
    train_features["horsepower"], train_labels,
    validation_split=0.2,
    verbose=0,
    epochs=100
)

Plot training progress:

In [ ]:
plt.plot(history.history['loss'], label='loss')
plt.plot(history.history['val_loss'], label='val_loss')
plt.xlabel('Epoch')
plt.ylabel('Error [MPG]')
plt.legend()
plt.grid(True)
plt.show()

Evaluate on the test set:

In [ ]:
test_results = {}
test_results['horsepower_model'] = horsepower_model.evaluate(
    test_features["horsepower"], test_labels, verbose=0)
test_results

Visualize predictions:

In [ ]:
x = tf.linspace(0.0, 250, 251)
y = horsepower_model.predict(x)

plt.scatter(train_features["horsepower"], train_labels, label='Data')
plt.plot(x, y, color='red', label='Prediction')
plt.xlabel('Horsepower')
plt.ylabel('MPG')
plt.legend()
plt.show()

## Linear regression with multiple inputs

Use all features in a linear model:

In [ ]:
linear_model = keras.Sequential([
    normalizer,
    layers.Dense(units=1)
])

linear_model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.1),
    loss='mean_absolute_error'
)

history = linear_model.fit(
    train_features, train_labels,
    validation_split=0.2,
    verbose=0,
    epochs=100
)

plt.plot(history.history['loss'], label='loss')
plt.plot(history.history['val_loss'], label='val_loss')
plt.xlabel('Epoch')
plt.ylabel('Error [MPG]')
plt.legend()
plt.grid(True)
plt.show()

test_results['linear_model'] = linear_model.evaluate(
    test_features, test_labels, verbose=0)
test_results

## Deep Neural Network (DNN) regression

We now build a deeper model with hidden layers to better capture nonlinearities.

### Single input DNN (Horsepower)

In [ ]:
def build_and_compile_model(norm_layer):
    model = keras.Sequential([
        norm_layer,
        layers.Dense(64, activation='relu'),
        layers.Dense(64, activation='relu'),
        layers.Dense(1)
    ])
    model.compile(loss='mean_absolute_error',
                  optimizer=keras.optimizers.Adam(0.001))
    return model

dnn_horsepower_model = build_and_compile_model(horsepower_normalizer)

history = dnn_horsepower_model.fit(
    train_features["horsepower"], train_labels,
    validation_split=0.2,
    verbose=0, epochs=100
)

plt.plot(history.history['loss'], label='loss')
plt.plot(history.history['val_loss'], label='val_loss')
plt.xlabel('Epoch')
plt.ylabel('Error [MPG]')
plt.legend()
plt.grid(True)
plt.show()

test_results['dnn_horsepower_model'] = dnn_horsepower_model.evaluate(
    test_features["horsepower"], test_labels, verbose=0)

x = tf.linspace(0.0, 250, 251)
y = dnn_horsepower_model.predict(x)

plt.scatter(train_features["horsepower"], train_labels, label='Data')
plt.plot(x, y, color='red', label='Prediction')
plt.xlabel('Horsepower')
plt.ylabel('MPG')
plt.legend()
plt.show()

### DNN with multiple inputs

In [ ]:
dnn_model = build_and_compile_model(normalizer)

history = dnn_model.fit(
    train_features, train_labels,
    validation_split=0.2,
    verbose=0, epochs=100
)

plt.plot(history.history['loss'], label='loss')
plt.plot(history.history['val_loss'], label='val_loss')
plt.xlabel('Epoch')
plt.ylabel('Error [MPG]')
plt.legend()
plt.grid(True)
plt.show()

test_results['dnn_model'] = dnn_model.evaluate(test_features, test_labels, verbose=0)
test_results

## Evaluate and compare models

In [ ]:
test_results

Check prediction performance:

In [ ]:
test_predictions = dnn_model.predict(test_features).flatten()

plt.scatter(test_labels, test_predictions)
plt.xlabel('True Values [MPG]')
plt.ylabel('Predictions [MPG]')
plt.axis('equal')
plt.axis('square')
plt.plot([0,50],[0,50], 'r')
plt.show()

error = test_predictions - test_labels
sns.kdeplot(error, shade=True)
plt.xlabel('Prediction Error [MPG]')
plt.ylabel('Density')
plt.show()

## Save and reload the model

In [ ]:
dnn_model.save('dnn_model')
reloaded = keras.models.load_model('dnn_model')

test_results['reloaded'] = reloaded.evaluate(test_features, test_labels, verbose=0)
test_results

We see that the reloaded model performs identically.

## Conclusion

- We explored linear and DNN models for regression.
- With proper normalization and a deeper architecture, the DNN model achieved better performance.
- We examined the loss, visualized predictions, and error distributions.
- We saved and reloaded the model.

You now have a foundation for performing regression tasks with neural networks in Python using TensorFlow/Keras.
```
```